In [54]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [55]:
# get dataset
dataset = pd.read_csv("Data/Apache_AllNumeric.csv")
dataset = dataset.sample(frac=1)
dataset_features = dataset.copy()
dataset_labels = dataset_features.pop('PERF')

In [56]:
# normalize dataset (MinMaxScale)
features_max = dataset_features.max()
labels_max = dataset_labels.max()
dataset_features /= features_max
dataset_labels /= dataset_labels.max()

In [57]:
# split dataset train (2/3) test (1/3)
x_train, x_test, y_train, y_test = train_test_split(dataset_features, dataset_labels, test_size=0.33)

In [76]:
# Define sequential model
model = keras.Sequential()
model.add(layers.Dense(16, activation='relu', kernel_regularizer=keras.regularizers.l1(0.001), input_shape=(8,)))
model.add(layers.Dense(8, activation='relu', kernel_regularizer=keras.regularizers.l1(0.001)))
model.add(layers.Dense(1))
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                loss="mean_squared_error",
                metrics=[keras.metrics.MeanSquaredError()])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [77]:
model.fit(x_train, y_train, epochs=50, batch_size=10, validation_split=0.2)


Epoch 1/50
11/11 [==============================] - 1s 22ms/step - loss: 0.3308 - mean_squared_error: 0.2643 - val_loss: 0.2580 - val_mean_squared_error: 0.1916
Epoch 2/50
11/11 [==============================] - 0s 7ms/step - loss: 0.2259 - mean_squared_error: 0.1595 - val_loss: 0.1816 - val_mean_squared_error: 0.1154
Epoch 3/50
11/11 [==============================] - 0s 6ms/step - loss: 0.1796 - mean_squared_error: 0.1134 - val_loss: 0.1491 - val_mean_squared_error: 0.0831
Epoch 4/50
11/11 [==============================] - 0s 5ms/step - loss: 0.1521 - mean_squared_error: 0.0862 - val_loss: 0.1332 - val_mean_squared_error: 0.0675
Epoch 5/50
11/11 [==============================] - 0s 6ms/step - loss: 0.1353 - mean_squared_error: 0.0697 - val_loss: 0.1217 - val_mean_squared_error: 0.0563
Epoch 6/50
11/11 [==============================] - 0s 6ms/step - loss: 0.1229 - mean_squared_error: 0.0575 - val_loss: 0.1121 - val_mean_squared_error: 0.0470
Epoch 7/50
11/11 [=====================

In [78]:
x = np.array([[1,0,1,1,1,1,1,0]])
y_pred = model.predict(x)
y_pred*labels_max

array([[802.38367]], dtype=float32)